In [1]:
import sys
project_root_path = '../..'
python_lib_path = project_root_path + '/python/lib'
sys.path.append(python_lib_path)
import commit_stats

In [2]:
import json
in_path = project_root_path + '/python/cfg/ds_pipeline.json'
config_map = json.loads(open(in_path, 'r').read())
config_map

cfg = config_map['language_analyzer'] # 99_problems

In [29]:
import time

start = time.time()
path = project_root_path + '/' + cfg['stats_dir']
all_commit_stats = commit_stats.load_commit_stats(cfg['stat_file_name'], path, 0, 800)
all_commit_stats[slice(0,2)]
end = time.time()
print(str(end - start) + " seconds, " + str((end - start)/60) + " minutes")

148.13861322402954 seconds, 2.468976887067159 minutes


In [30]:
import time

start = time.time()
ft_dicts = []
ft_dict_dict = {}
for stats in all_commit_stats:
    for file_type in stats['fileTypes']:
        commit_ft_dict = stats['fileTypes'][file_type]
        if file_type not in ft_dict_dict:
            ft_dict = {}
            ft_dicts.append(ft_dict)
            ft_dict_dict[file_type] = ft_dict
            ft_dict['type'] = file_type
            ft_dict['lines'] = 0
            ft_dict['files'] = 0
            ft_dict['commits'] = 0
            ft_dict['repos'] = {}
        ft_dict = ft_dict_dict[file_type]
        ft_dict['commits'] += 1
        ft_dict['lines'] += commit_ft_dict['inserts']
        ft_dict['files'] += commit_ft_dict['occurrences']
        #seen_in = commit_ft_dict['seen_in']
        #for project in seen_in:
        #    ft_dict['repos'][project] = 1

end = time.time()
print(str(end - start) + " seconds, " + str((end - start)/60) + " minutes")


3.380836248397827 seconds, 0.056347270806630455 minutes


In [31]:
import time

start = time.time()
by_commit_count = sorted(ft_dicts, key=lambda d: d['commits'])
by_commit_count.reverse()
# if your odds of being in a commit are < 1/1000, you don't matter
main_type_stats = [stats for stats in by_commit_count
                   if stats['commits'] > (len(all_commit_stats) / 1000)]
print(len(main_type_stats))

end = time.time()
print(str(end - start) + " seconds, " + str((end - start)/60) + " minutes")

main_type_stats


98
0.034490346908569336 seconds, 0.0005748391151428223 minutes


[{'type': 'py',
  'lines': 27648204,
  'files': 921419,
  'commits': 357039,
  'repos': {}},
 {'type': 'md',
  'lines': 24766465,
  'files': 1039999,
  'commits': 317841,
  'repos': {}},
 {'type': 'java',
  'lines': 86000876,
  'files': 2441244,
  'commits': 299269,
  'repos': {}},
 {'type': 'noexttext',
  'lines': 18588758,
  'files': 684659,
  'commits': 260389,
  'repos': {}},
 {'type': 'js',
  'lines': 65504212,
  'files': 1153526,
  'commits': 252311,
  'repos': {}},
 {'type': 'c',
  'lines': 37919135,
  'files': 639126,
  'commits': 228314,
  'repos': {}},
 {'type': 'h',
  'lines': 26756497,
  'files': 632184,
  'commits': 191323,
  'repos': {}},
 {'type': 'json',
  'lines': 104359884,
  'files': 835085,
  'commits': 190216,
  'repos': {}},
 {'type': 'cpp',
  'lines': 20508792,
  'files': 525755,
  'commits': 186001,
  'repos': {}},
 {'type': 'go',
  'lines': 116687777,
  'files': 857038,
  'commits': 96240,
  'repos': {}},
 {'type': 'rb',
  'lines': 5799205,
  'files': 300819,
 

In [ ]:
import pandas as pd

main_types = [stat['type'] for stat in main_type_stats]
main_types.index('json')
rows = []
for commit_stats in all_commit_stats:
    row = [0]*len(main_types)
    rows.append(row)
    for file_type in commit_stats['fileTypes']:
        commit_ft_dict = commit_stats['fileTypes'][file_type]
        if file_type in main_types:
            position = main_types.index(file_type)
            row[position] = commit_ft_dict['inserts']
type_df = pd.DataFrame(rows, columns=main_types) # index=commit_ids,
type_df

In [ ]:
type_corr = type_df.corr()
print(str(type_corr))

In [ ]:
for i in range(len(main_types)):
    for j in range(len(main_types)):
        if i < j:
            val = type_corr.iat[i, j]
            if val > 0.2:
                print(main_types[i] + ", " + main_types[j] + ": " + str(val))